In [1]:
import json
import lxml.html
import httpx
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_Chicago_alderpersons_since_1923'

def scrape_wikipedia_table(url):

    
    # URL
    url = 'https://en.wikipedia.org/wiki/List_of_Chicago_alderpersons_since_1923'

    # HTML Content
    response = httpx.get(url)
    root = lxml.html.fromstring(response.content)
    
    # Find all tables 
    tables = root.xpath('//table[contains(@class, "wikitable sortable")]')

    #Store Extracted data
    all_data = []

    # Iterate through each ward
    for table in tables:

        # Extract table headers to determine column indices
        header_indices = {header.text_content().strip(): idx for idx, header in enumerate(table.xpath('.//tr/th'))}
        columns_of_interest = ["Alderperson", "Term in office", "Party", "Notes"]

        for row in table.xpath('.//tr[position()>1]'):
            
            # Extract both <td> and <th scope="row"> elements
            cells = row.xpath('.//td | .//th[@scope="row"]')
            row_data = [
                cells[header_indices[col]].text_content().strip() 
                if header_indices.get(col) is not None and header_indices[col] < len(cells) else '' 
                for col in columns_of_interest
            ]
            all_data.append(row_data)

        # Print the data for the first table and break
        print("First Table Data:")
        for row in all_data:
            print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]}")
        break  # Stop after processing the first table

    # Handle any None or empty values
    for ward in all_data:
        for i in range(len(ward)):
            if ward[i] is None or ward[i] == '':
                ward[i] = 'N/A'

    # Save the data to a JSON file
    with open("all_data.json", "w") as jd:
        json.dump(all_data, jd, indent=1)

    return all_data


# Call the function and store the scraped data
scraped_data = scrape_wikipedia_table(url)

print("Scraping completed.")


NameError: name 'url' is not defined